<a href="https://colab.research.google.com/github/heejookang-sguniv/NLP-/blob/main/120220107%20%EA%B0%95%ED%9D%AC%EC%A3%BC%20XLNET%20%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Introduction**

많은 양의 코퍼스로 Language Modeling 에 대한 Pre-training을 진행하고 특정 테스크로 Fine-tuning을 진행하는 방법은 BERT 이후로 NLP 문제를 풀기위한 정석과 같은 방법이 되었다. XLNet에서는 BERT와 같이 Masked Language Modeling을 objective로 하는 Autoencoder(AE)방식과 GPT와 같이 AutoRegreesive(AR) Language Modeling 을 objective로 하는 방식의 장점을 유지하면서 단점을 보완하는 새로운 학습 방식을 제안한다. 

AR 방식과 AE 방식은 Pre-trianing의 대표적인 objective를 말한다. AE는 어떤 값을 그대로 복원해내는 것에 목적이 있으며, AR에서는 기존의 언어모델과 비슷하게 $x_T$ 를 예측하기 위해서 $x_1, x_2, ... x_{T-1}$을 이용해 $x_T$가 등장할 확률을 가장 최대화할 수 있게 만들어 주어야 한다. 


AE의 단점은 input sequence에서 특정 부분에 [Mask]를 적용하여 input sequence를 복원하도록 학습하는데, pre-trian에 사용하는 [Mask]가 fine-tune 단계에서는 사용되지 않아 pre-train과 fine-tune간에 불일치 문제가 존재한다. 또한 AE에서는 predicted token이 input sequence에서 [Mask]처리되어 있기 때문에 AR language modeling에서 곱의 규칙을 통해 얻을 수 있는 joint probability를 계산할 수 없다. joint probability를 사용하지 않는다는 것은 input sequence로부터 predicted token들이 서로 독립적이라고 추정하게 되며, 이는 고차원(high-order), 장거리 의존성(long-term dependency)이 있는 자연어에서 *지나치게 단순화*된다고 불 수 있다. 
AR의 단점은 현재 token인 $x_T$을 예측하기 위해 &1부터 T-1&까지 token의 conditional probability를 통해 확률분포를 추정하기 때문에 양방향 context를 모델링하는데는 적합하지 않다. 

때문에 기존의 language pre-training의 장단점에 직면하여 AR language modeling과 AE를 최대한 활용하는 generalized autoregressive method(즉, 양방향 정보를 이용할 수 있는 AR Language Modeling 학습법)인 *XLNet*을 제안한다. 

### **1. AR, AE Language Modeling**

Language Model은 특정 토큰들의 시퀀스 $[x_1, x_2, x_3, ..., x_T]$에 확률$P(x_1, x_2, x_3, ..., x_T)$을 할당하는 확률 분포 모델이다. 주로 언어 자체의 분포, 즉 "주어진 시퀀스가 얼마나 자연스러운가"를 학습하고 비지도학습으로 진행할 수 있기 때문에 pre-training에 많이 이용되고 있다. 

###**1.1 Auto Regressive(AR)**
AR Language modeling은 확률을 할당하기 위해 주어진 텍스트 시퀀스 $X = [x_1, x_2, x_3, ..., x_T]$에 대한 확률 분포를 $p(X) = \prod_{t=1}^{T} p(x_t | x_{<t}) $ 와 같이 특정 방향(정방향/역방향)의 곱으로 분해한다. Neural Network는 이러한 조건부 확률 분포를 학습한다. 
  
*input sequence* : $X = (x_{1}, x_{2}, ..., x_{T})$

*forward likelihood* : $p(X) = \prod_{t=1}^{T}p(x_{t}|x_{<t})$

*training obgective(forward)* : $max_{\theta}log$  $p_{\theta}(x) = max_{\theta} \sum_{t=1}^{T} log$ $p(x_{t}|x_{<t})$


즉, $[x_1, x_2, x_3, ..., x_T]$ 가 주어졌을 때, $p(x_2 | x_1)p(x_3 | x_1, x_2) ...$의 확률 분포를 모델링하도록 학습을 진행한다. 이 방법은 Language Modeling이라는 목적을 달성하기 위해, 어떠한 가정 없이 $p(X)$를 조건부 확률의 곱으로 분해하여 표현할 수 있는 장점이 있다. 하지만 특정 방향의 context만 인코딩하도록 학습되기 때문에 pre-training 방법으로 이용할 시, 양방향 context를 고려하는 테스크들에 효과가 떨어지는 단점이 있다. 

###**1.2 Auto Encoding(AE)**
Auto Encoding은 주어진 입력을 그대로 예측하는 문제를 푸는데, 주로 차원 축소등을 목적으로이용된다. BERT에서 사용된 시퀀스의 토큰을 일정 확률로  [MASK]로 치환한 뒤 이를 원래 토큰으로 복원하는 방법은 노이즈가 섞여 있는 입력을 원래의 입력으로 예측하는 Denoising AE라 볼 수 있다.  

*input sequence* : $\overline{X} = (x_1, x_2, ..., x_T)$

*corrupted input* : $\widehat{X} = (x1, [MASK], ..., x_T)$

*likelihood* : $p(\overline{X} | \widehat{X}) \approx \prod_{t=1}^{T}p(x_{t}|\widehat{x})$ 

*training objective* : $max_{\theta}log$ $p(\overline{X} | \widehat{X}) = max_{\theta} \sum_{t=1}^{T} m_{t}log$  $p(x_t | \widehat{x})$ 

즉, $[x_1, [MASK]_{1}, x_3, x_4, [MASK]_{2}, ..., x_T]$ 가 주어졌을 때, 원래 토큰 분포를 에측하는 $P([MASK]_{1}| x_1, x_3, x_4, [MASK]_{2}, ...)P([MASK]_{2}| x_1, [MASK]_{1},x_3, x_4,... )$를 학습한다. 이 방법은 AR에 비해 양방향 컨텍스트를 고려하여 학습할 수 있다는 장점이 있다. 하지만 [MASK]는 기존 언어 분포에 없는 노이즈이고, pre-training 시에만 이용되기 때문에 다른 테스크들과 불일치 문제가 발생한다. 또한 각 [mask] 들이 위이 근사식처럼 독립적으로 예측되어 [MASK]토큰들 사이의 의존관계를 학습할 수 없다. 

### **2. XLNet**

위에서 제시한 두 가지 Language Modeling의 단점을 보완하고 장점을 살릴 수 있는 새로운 objective와 이를 학습할 수 있는 모델구조인 XLNet을 제시한다. 


*   기존의 AR model들은 previous tokens를 통해 modeling을 하였지만, XLNet은 입력 sequence의 token들로 만들 수 있는 모든 순열에 대해 log-likelihood를 최대화 할 수 있게 학습한다. 이를 통해 각 위치의 context는 왼쪽과 오른쪽의 token으로 구성될 수 있으며, 각 위치는 양방향 문맥을 capture하여 학습한다. 
*   XLNet은 generalized AR language model이기 때문에 pre-train과 fine-tune 간에 차이가 존재하지 않는다. 곱의 법칙을 통해 token들의 joint probability를 계산하기 때문에 BERT에서 발생한 token들을 독립적으로 추정하게 되는 문제를 발생시키지 않는다. 
*   추가적으로 XLNet은 pre-training을 위한 architecture design을 개선한다. 

    1. **segment recurrence mechanism**과 Transformer-XL의 **relative encoding scheme**을 pre-training에 적용하여 길이가 긴 sequence에 대해서 성능을 향상시킨다. 
    2. XLNet과 같은 permutation-based language modeling에 Transformer-XL architecture를 naive하게 적용하는 것은 어렵기 때문에 Transformer-XL network를 reparameterize하는 방법을 제안한다. 

###**2.1 Obective : Permutation Language Modeling**

특정한 가정 없이 모델링을 표현할 수 있는 AR 모델의 이점과 양방향의 컨텍스트를 이용할 수 있는 AE 모델의 이점을 모두 살리면서 나머지 단점/한계점들을 보완하는 **Permutation Language Modeling**이라는 새로운 objective를 제시한다. 

길이 *T*의 시쿼스 $X = [x_1, x_2, ...,x_T]$가 주어졌을 때, 시퀀스를 나열할 수 있는 모든 순서의 집합($Z_{t}$) - 순열(Permutation)은 총 T!개 만들 수 있다. 이때 다음 토큰을 예측하는 AR formula를 전체 permutation 집합 $Z_{t}$에서 진행하는 것이 XLNet에서 추구하는 objective function에 해당한다. 아래의 식과 같이 $Z_T$에 속해있는 모든 순서들을 고려하여 AR 방식으로 모델링을 진행하고, 각 순서에 대한 log likelihood 기댓값을 최대화한다. 

*likelihood* :$E_{z \sim Z_{T}}[\prod_{t=1}^{T}p_{\theta}(x_{z_{t}}|x_{z<t})]$

*training objective*: $max_{\theta} E_{z \sim Z_{T}}[\sum_{t=1}^{T}log$ $p_{\theta}(x_{z_{t}}|x_{z<t})]$

예를 들어 총 6개의 token이 있을 때,
길이가 6인 모든 순서들의 집합은 $Z_6 = [x_1,x_2,x_3,x_4,x_5,x_6], [x_2,x_3,x_4,x_5,x_6,x_1], ..., [x_6,x_5,x_4,x_3,x_2,x_1]$과 같이 주어지며, 각 순서들에 대한 AR 모델링을 고려할 수 있다. 

$z=[x_1,x_2,x_3,x_4,x_5,x_6]$인 경우 기존 시퀀스의 순서 그대로 진행하여 $\prod_{t=1}^{T}p_{\theta}(x_{z_{t}}|x_{z<t})$는 $p(x_2|x_1)p(x_3|x_1,x_2)...p(x_6|x_1,x_2,x_3,x_4,x_5)$ 가 되고, $z = [x_2,x_3,x_4,x_5,x_6,x_1]$인 경우에는 $p(x_3|x_2)p(x_4|x_2,x_3)...p(x_1|x_2,x_3,x_4,x_5,x_6)$가 된다. 이때, $p(x_1|x_2,x_3,x_4,x_5,x_6)$가 의미하는 것은 2번째 위치에 $x_2$토큰, 3번째 위치에 $x_3$ 토큰, 4번째 위치에 $x_4$라는 토큰, ... 이 주어졌을 때, 첫번째 위치에 $x_1$이라는 토큰이 올 확률을 나타낸다. 
즉, permutation 집합을 통해 다양한 sequence를 고려하게 되고, 다양한 sequence를 AR objective function에 대입함으로써 특정 token에 대해서 양방향 context를 고려할 수 있다. 

이때 주의해야 할 점은 시퀀스 자체의 순서를 섞는 것이 아니라 $p(x)$를 조건부 확률들의 곱으로 분리할 때 이 순서만 섞는다는 것이다. 즉, 모델은 기존 시퀀스의 토큰들의 절대적 위치를 알 수 있다. 

시퀀스 길이 $T$에 대해 가능한 순열의 갯수는 $T!$개를 갖기 때문에, 하나의 텍스트 $[x_1,x_2,...,x_T]$에 대해 순열($Z_T$)의 모든 경우를 고려하는 것은 불가능하다. 따라서 하나의 텍스트 시퀀스에 대해 하나의 permutation 순서($z$)를 샘플링하고 해당 순서에 대해 $p_{\theta}(x)$를 $\prod_{t=1}^{T}p_{\theta}(x_{z_{t}} | x_{z<t})$로 분해한다. 하지만 모델의 파라미터($\theta$)는 학습하는 동안 모든 순서에 대해 공유되므로, 많은 양의 데이터를 거치면 모든 순서를 고려한다고 볼 수 있다. 따라서 모델은 이 과정에서 어떠한 근사 없이 양방향 컨텍스트를 볼 수 있는 능력을 갖게 된다. 

###**2.2 Architectur:Two-Stream Self-Attention for Target-Aware Representations**

앞에서 새롭게 정의했던 objective function 자체는 standard transform에서 작동하지 않는다. 따라서 Transformer에서 XLNet의 objective function을 적용하기 위해 **Target-Aware Representation**을 제안했다.
기존의 AR model에서는 학습시 방향이 일정하기 때문에 예측할 token이 아주 명확하지만, XLNet의 AR model에서는 학습시 permutation을 진행하기 때문에 예측할 token이 명확하지 않다. 예를 들어 총 4개의 token으로 이루어진 sequence가 있을 때, $z_1 = [x_1,x_2, x_3, x_4]$, $z_2 = [x_1, x_2, x_4, x_3]$ 과 같이 permutation order가 구성되어 있다. 이때, $z_1$에서 $x_3$을 예측하기 위해서는 $x_1$과 $x_2$를 사용해서 모델링을 진행한다($p(x_3|x_1,x_2)$). $z_2$에서 $x_4$를 예측할 때도 $z_1$과 마찬가지로 $x_1$과 $x_2$를 사용한다($p(x_3|x_1,x_2)$). 즉, 동일한 representation으로 다른 target을 맞추어야 하기 때문에 학습할 시 예측할 token이 명확하지 않은 문제가 발생한다. 이를 해결하기 위해 예측할 representation과 타켓하는 토큰을 함께 학습하고자 하는 것이 바로 *Target-Aware Representation*이다. 즉, 모델의 입력으로 토큰의 위치정보($z_t$)를 추가적으로 제공하여 아래의 식과 같이 예측할 토큰에 대한 확률 분포를 계산한다. 

$p_{\theta}(X_{z_{t}}= x |\ x_{z<x}) = \frac{exp(e(x)^Tg_{\theta}(x_{z<t}, z_{t}))}{\sum_{x^{'}}exp(e(x^{'})^{T}g_{\theta}(x_{z<t}, z_{t}))}$ 

###Two-Stream Self-Attention

위의 방법으로 예측할 토큰에 대한 모호성은 해결하였다. 이제, 기존의 $h_{\theta}(x_{z<t})$를 계산하던 모델을 $g_{\theta}(x_{z<t}, z_{t})$을 계산하도록 변경하는 문제가 남아있다. 이를 위해 현재 예측하고자 하는 시점 $z_{t}$는 주변 컨텍스트($x_{z<t}$)와 attention을 통해 정보를 축적해 나가는 방식을 제시한다. 두가지 제약 조건을 통해 이 방법을 실현하는데, 기존의 transformer 구조를 이용하면 이들 사이에 모순이 생긴다. 

1. 토큰 $x_{z_{t}}$를 예측하기 위해 $g_{\theta}(x_{z<t}, z_{t})$는 위치에 대한 정보 $z_t$만 이용하고 컨텐츠(단어 자체에 대한 정보) $x_{z_t}$를 이용하면 안된다. 단어 자체에 대한 정보를 제공하면, 정답 단어를 제공하는 꼴이 되기 때문에, 의미 없는 쉬운 문제가 된다. 
2. 다음 토큰들($x_{z>t}$)을 예측할 때에는 $g_{\theta}(x_{z<t}, z_{t})$는 컨텐츠에 대한 정보를 제공하기 위해 $x_{z_{t}}$를 인코딩 해야한다.  

위의 두 조건은 특정 시점에서 하나의 hidden state을 인코딩하는 기존의 transformer 구조에서는 서로 모순이다. 따라서 두 가지 hidden state들을 이용하는 새로운 방법을 제시한다. 


*   **content representation $h_{\theta}(x_{z<t}):$** $z \leq t$ 시점의 정보($x$)를 이용하여 컨텍스트와 $x_{z_t}$에 대한 정보를 함께 인코딩한다. 이는 기존 transformer의 hidden state들과 비슷한 역할을 하게 되며, 이를 통해 2번 제약 조건을 만족시킨다. 
*   **Query representation $g_{\theta}(x_{z<t}, z_{t}):$** $z < t$ 시점의 정보($x$)와 $z_t$시점의 위치만을 이용하여 컨텍스트에 대한 정보를 인코딩한다. 이를 통해 1번 제약 조건을 만족시킨다. 

두 stream은 파라미터($\theta$)를 공유하며 학습을 진행한다. fine-tuning 을 진행할 때에는 Query 스트림을 제거하고 content 스트림을 transformer와 동일하게 이용한다. 



### _create_data
함수에서 data/example 에 있는 모든 *.txt 파일을 읽어와 줄 단위로 loop를 돌며, preprocess_text 함수를 통해 sentence piece로 토크나이징한다. 
여기서 input_shards 변수는 list 타입으로 각 elemnet는 한 파일의 (input_data, sent_ids)이 저장된다. input_data는 모든 line의 token이다. sent_ids는 각 line을 구분해주기 위한 list로 line이 바뀔 때 마다 True, False로 바꿔가며 line의 경계를 구분해준다. 
이렇게 만들어진 모든 텍스트 파일의 input_data와 sent_ids는 input_data = np.comcatenate(input_data_list)와 sent_ids = np.concatenate(sent_ids_list)를 통해 하나의 list로 합쳐진다. 

In [ ]:
def _create_data(idx, input_paths):
  # Load sentence-piece model
  sp = spm.SentencePieceProcessor()
  sp.Load(FLAGS.sp_path)

  input_shards = []
  total_line_cnt = 0
  for input_path in input_paths:
    input_data, sent_ids = [], []
    sent_id, line_cnt = True, 0
    tf.logging.info("Processing %s", input_path)
    for line in tf.gfile.Open(input_path):
      if line_cnt % 100000 == 0:
        tf.logging.info("Loading line %d", line_cnt)
      line_cnt += 1

      if not line.strip():
        if FLAGS.use_eod:
          sent_id = not sent_id
          cur_sent = [EOD_ID]
        else:
          continue
      else:
        if FLAGS.from_raw_text:
          cur_sent = preprocess_text(line.strip(), lower=FLAGS.uncased)
          cur_sent = encode_ids(sp, cur_sent)
        else:
          cur_sent = list(map(int, line.strip().split()))

      input_data.extend(cur_sent)
      sent_ids.extend([sent_id] * len(cur_sent))
      sent_id = not sent_id

    tf.logging.info("Finish with line %d", line_cnt)
    if line_cnt == 0:
      continue

    input_data = np.array(input_data, dtype=np.int64)
    sent_ids = np.array(sent_ids, dtype=np.bool)

    total_line_cnt += line_cnt
    input_shards.append((input_data, sent_ids))

  tf.logging.info("[Task %d] Total number line: %d", idx, total_line_cnt)

  tfrecord_dir = os.path.join(FLAGS.save_dir, "tfrecords")

  filenames, num_batch = [], 0

  # Randomly shuffle input shards (with a fixed but distinct random seed)
  np.random.seed(100 * FLAGS.task + FLAGS.pass_id)

  perm_indices = np.random.permutation(len(input_shards))
  tf.logging.info("Using perm indices %s for pass %d",
                  perm_indices.tolist(), FLAGS.pass_id)

  input_data_list, sent_ids_list = [], []
  prev_sent_id = None
  for perm_idx in perm_indices:
    input_data, sent_ids = input_shards[perm_idx]
    # make sure the `send_ids[0] == not prev_sent_id`
    if prev_sent_id is not None and sent_ids[0] == prev_sent_id:
      sent_ids = np.logical_not(sent_ids)

    # append to temporary list
    input_data_list.append(input_data)
    sent_ids_list.append(sent_ids)

    # update `prev_sent_id`
    prev_sent_id = sent_ids[-1]

  input_data = np.concatenate(input_data_list)
  sent_ids = np.concatenate(sent_ids_list)

  file_name, cur_num_batch = create_tfrecords(
      save_dir=tfrecord_dir,
      basename="{}-{}-{}".format(FLAGS.split, idx, FLAGS.pass_id),
      data=[input_data, sent_ids],
      bsz_per_host=FLAGS.bsz_per_host,
      seq_len=FLAGS.seq_len,
      bi_data=FLAGS.bi_data,
      sp=sp,
  )

### _local_perm

아래 코드의 index = tf.random_shuffle(index)이 Permutation of the factorization order를 만드는 코드이다. 논문과 마찬가지로 seq_len이 512라면, 512! 개의 permutation factorization order를 만들지 않고, 한 번만 샘플링한다. 

In [ ]:
def _local_perm(inputs, targets, is_masked, perm_size, seq_len):
	# Generate permutation indices
  index = tf.range(seq_len, dtype=tf.int64)
  index = tf.transpose(tf.reshape(index, [-1, perm_size]))
  index = tf.random_shuffle(index)
  index = tf.reshape(tf.transpose(index), [-1])

_local_perm 함수를 계속해서 보았을 때, non_func_tokens 변수의 논리 연산자를 풀면 아래와 같다. 

non_mask_tokens = $(\sim(sep \vee cls))\wedge (\sim$(is_masked)) 

= $\sim sep \wedge \sim cls \wedge \sim$is_masked

masked_or_func_tokens = $\sim$ non_mask_tokens

= ($sep \wedge cls) \vee$ (is_masked)

In [ ]:
non_func_tokens = tf.logical_not(tf.logical_or(
      tf.equal(inputs, SEP_ID),
      tf.equal(inputs, CLS_ID)))

non_mask_tokens = tf.logical_and(tf.logical_not(is_masked), non_func_tokens)
masked_or_func_tokens = tf.logical_not(non_mask_tokens)

rev_index = tf.where(non_mask_tokens, smallest_index, index)에서 ~sep이고 ~cls이고 ~is_masked인 경우는 -1 값을, 아닌 경우는 permutation factorization order의 index를 넣어준다. 

target_tokens = $((sep \wedge cls)\wedge$ (is_masked)) $\wedge (\sim (sep \wedge cls))$

= $(sep \wedge cls)\wedge(\sim(sep \wedge cls)) \vee$(is_masked)$\wedge (\sim(sep \wedge cls))$ 

target tokens의 경우, sep 와 또는 cls이 낭니고, is_masked 인 것을 나타낸다. 즉, 실제로 mask해야하는 토큰들이다. 

In [ ]:
# Set the permutation indices of non-masked (& non-funcional) tokens to the
# smallest index (-1):
# (1) they can be seen by all other positions
# (2) they cannot see masked positions, so there won"t be information leak
smallest_index = -tf.ones([seq_len], dtype=tf.int64)
rev_index = tf.where(non_mask_tokens, smallest_index, index)

# Create `target_mask`: non-funcional and maksed tokens
# 1: use mask as input and have loss
# 0: use token (or [SEP], [CLS]) as input and do not have loss
target_tokens = tf.logical_and(masked_or_func_tokens, non_func_tokens)
target_mask = tf.cast(target_tokens, tf.float32)

아래의 코드는 실제 permutation factorization order에 맞는 perm_mask를 만드는 코드이다. 
self_rev_index = tf.where(target_tokens, rev_index, rev_index + 1)를 통해 실제로 마스킹해야하는 토큰들은 rev_index을 살리고, 그렇지 않으면 rev_index +1 값으로 변경시켜준다. 앞의 -1값은 0으로 업데이트된다. 

perm_mask = tf.logical_and(self_rev_index[:, None] <= rev_index[None, :], masked_or_func_tokens)에서 i<=j 이고, j가 masked_or_func_tokens인 경우에만 1을, 그렇치 않으면 0인 perm_mask를 만든다. 이렇게 만들어진 perm_mask는 후에 attn_mask_h로 쓰인다. 

In [ ]:
# Create `perm_mask`
# `target_tokens` cannot see themselves
self_rev_index = tf.where(target_tokens, rev_index, rev_index + 1)

# 1: cannot attend if i <= j and j is not non-masked (masked_or_func_tokens)
# 0: can attend if i > j or j is non-masked
perm_mask = tf.logical_and(
    self_rev_index[:, None] <= rev_index[None, :],
    masked_or_func_tokens)
perm_mask = tf.cast(perm_mask, tf.float32)

논문의 two stream relative attention이 일어나는 부분은 아래의 코드와 같다. 

In [ ]:
def two_stream_rel_attn(h, g, r, mems, r_w_bias, r_r_bias, seg_mat, r_s_bias,
                        seg_embed, attn_mask_h, attn_mask_g, target_mapping,
                        d_model, n_head, d_head, dropout, dropatt, is_training,
                        kernel_initializer, scope='rel_attn'):
  """Two-stream attention with relative positional encoding."""
...
	# content based attention score
    if mems is not None and mems.shape.ndims > 1:
      cat = tf.concat([mems, h], 0)
    else:
      cat = h

		# content-based key head
    k_head_h = head_projection(
        cat, d_model, n_head, d_head, kernel_initializer, 'k')

    # content-based value head
    v_head_h = head_projection(
        cat, d_model, n_head, d_head, kernel_initializer, 'v')

    # position-based key head
    k_head_r = head_projection(
        r, d_model, n_head, d_head, kernel_initializer, 'r')

		##### h-stream
    # content-stream query head
    q_head_h = head_projection(
        h, d_model, n_head, d_head, kernel_initializer, 'q')

cat = tf.concat([mems, h], 0)으로 캐싱된 mem이 있다면 h(input_k)와 concatenate 한다. head_projection를 4번 호출하여, 각각 k_head_h, v_head_h, k_head_r, h-stream을 head projection하고 있다. head_projection 함수는 hidden state를 head로 projection시키는 역할을 한다. 

아래의 수식에 대한 구현 코드는 아래의 코드와 같다. 

$h_{z_{t}}^{(m)} <- Attention(Q=h_{z_{t}}^{(m-1)}, KV = h_{z_{\leq t}}^{(m-1)};\theta)$ (content stream: use both $z_{t}$ and $x_{z_{t}}$)

In [ ]:
# core attention ops
attn_vec_h = rel_attn_core(
    q_head_h, k_head_h, v_head_h, k_head_r, seg_embed, seg_mat, r_w_bias,
    r_r_bias, r_s_bias, attn_mask_h, dropatt, is_training, scale)

# post processing
output_h = post_attention(h, attn_vec_h, d_model, n_head, d_head, dropout,
                          is_training, kernel_initializer)

if target_mapping is not None: 을 만족하므로 아래 수식을 구현하고, 

$g_{z_{t}}^{(m)} <- Attention(Q = g_{z_{t}}^{(m-1)}, KV = h_{z_{<t}}^{(m-1)};\theta)$, (query stream: use $z_{t}$ but cannot see $x_{z_{t}})

_local_perm 함수에서 만들었던 target_mapping을 사용한다. attn_vec_g = tf.einsum('lbnd, mlb -> mbnd', attn_vec_g, target_mapping)

In [ ]:
with tf.variable_scope(scope, reuse=True):
    ##### g-stream
    # query-stream query head
    q_head_g = head_projection(
        g, d_model, n_head, d_head, kernel_initializer, 'q')

# core attention ops
    if target_mapping is not None:
      q_head_g = tf.einsum('mbnd,mlb->lbnd', q_head_g, target_mapping)
      attn_vec_g = rel_attn_core(
          q_head_g, k_head_h, v_head_h, k_head_r, seg_embed, seg_mat, r_w_bias,
          r_r_bias, r_s_bias, attn_mask_g, dropatt, is_training, scale)
      attn_vec_g = tf.einsum('lbnd,mlb->mbnd', attn_vec_g, target_mapping)
    else:
      attn_vec_g = rel_attn_core(
          q_head_g, k_head_h, v_head_h, k_head_r, seg_embed, seg_mat, r_w_bias,
          r_r_bias, r_s_bias, attn_mask_g, dropatt, is_training, scale)

    # post processing
    output_g = post_attention(g, attn_vec_g, d_model, n_head, d_head, dropout,
                              is_training, kernel_initializer)